In [1]:
def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]

In [2]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np

# global variables
api_key = 'RGAPI-08e8b384-d01b-4a60-83a2-c05dadc8f8ab'
watcher = LolWatcher(api_key)
my_region = 'euw1'


In [27]:
# fetch matchlist by puuid
me = watcher.summoner.by_name(my_region, 'kelody')
# me = watcher.summoner.by_name(my_region, 'daimox')
my_matches = watcher.match.matchlist_by_puuid("EUROPE", me['puuid'])
matches = []
matchTimeLine = []
participants = []


In [28]:
# fetch game, participant and match timeline and transform into dataframe
for matchId in my_matches:
    match_detail = watcher.match.by_id("EUROPE", matchId)
    timeline = watcher.match.timeline_by_match('EUROPE', matchId)

    match = {}
    match['matchId'] = matchId
    match['gameDuration'] = match_detail['info']['gameDuration']
    match['gameType'] = match_detail['info']['gameType']
    match['gameVersion'] = match_detail['info']['gameVersion']
    matches.append(match)

    for index, row in enumerate(match_detail['info']['participants']):
        participants_row = {}
        participants_row['matchId'] = matchId
        participants_row['participant'] = index + 1
        participants_row['summonerName'] = row['summonerName']
        participants_row['teamPosition'] = row['teamPosition']
        participants_row['championName'] = row['championName']
        participants_row['kda'] = f"{row['kills']}/{row['deaths']}/{row['assists']}"
        participants_row['champLevel'] = row['champLevel']
        participants_row['totalDamageToChampions'] = row['totalDamageDealtToChampions']
        participants_row['totalDamageTaken'] = row['totalDamageTaken']
        participants_row['totalHeal'] = row['totalHeal']
        participants_row['totalMinionsKilled'] = row['totalMinionsKilled']
        participants_row['goldEarned'] = row['goldEarned']
        participants_row['summoner1'] = row['summoner1Id']
        participants_row['summoner2'] = row['summoner2Id']
        participants_row['item0'] = row['item0']
        participants_row['item1'] = row['item1']
        participants_row['item2'] = row['item2']
        participants_row['item3'] = row['item3']
        participants_row['item4'] = row['item4']
        participants_row['item5'] = row['item5']
        participants_row['item6'] = row['item6']
        participants_row['win'] = row['win']
        participants.append(participants_row)

    for row in timeline['info']['frames']:
        # frameCounter = 1
        # frame = row["{:02d}".format(frameCounter)]
        # frameCounter += 1
        for frameRow in row['events']:
            timeline_row = {}
            timeline_row['matchId'] = matchId
            eventCounter = 0
            # eventFrame = frameRow["{:02d}".format(eventCounter)]
            eventFrame = frameRow
            eventCounter += 1

            if eventFrame['type'] == 'CHAMPION_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['victimId'] = eventFrame['victimId']
                if 'assistingParticipantIds' in eventFrame:
                    timeline_row['assistingParticipantIds'] = eventFrame['assistingParticipantIds']
                if 'bounty' in eventFrame:
                    timeline_row['bounty'] = eventFrame['bounty']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'BUILDING_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['teamId'] = eventFrame['teamId']
                timeline_row['buildingType'] = eventFrame['buildingType']
                timeline_row['laneType'] = eventFrame['laneType']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'ITEM_PURCHASED':
                timeline_row['type'] = eventFrame['type']
                timeline_row['participantId'] = eventFrame['participantId']
                timeline_row['itemId'] = eventFrame['itemId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'WARD_PLACED':
                timeline_row['type'] = eventFrame['type']
                timeline_row['wardType'] = eventFrame['wardType']
                timeline_row['creatorId'] = eventFrame['creatorId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'WARD_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['wardType'] = eventFrame['wardType']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'LEVEL_UP':
                timeline_row['type'] = eventFrame['type']
                timeline_row['participantId'] = eventFrame['participantId']
                timeline_row['level'] = eventFrame['level']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'ELITE_MONSTER_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['teamId'] = eventFrame['killerTeamId']
                if 'assistingParticipantIds' in eventFrame:
                    timeline_row['assistingParticipantIds'] = eventFrame['assistingParticipantIds']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                timeline_row['monsterType'] = eventFrame['monsterType']
                matchTimeLine.append(timeline_row)


participantsDf = pd.DataFrame(participants)
matchesDf = pd.DataFrame(matches)
matchTimelineDf = pd.DataFrame(matchTimeLine)
print(participantsDf)
print(matchesDf)
print(matchTimelineDf)


             matchId  participant      summonerName teamPosition  \
0    EUW1_5683109745            1               rzα          TOP   
1    EUW1_5683109745            2  Endstation Knast       JUNGLE   
2    EUW1_5683109745            3  Chinesefastcombo       MIDDLE   
3    EUW1_5683109745            4     LoLinator2012       BOTTOM   
4    EUW1_5683109745            5  xXUnForgiVablEXx      UTILITY   
..               ...          ...               ...          ...   
195  EUW1_5668562723            6  Blade of Shibuya          TOP   
196  EUW1_5668562723            7        Misterjony       JUNGLE   
197  EUW1_5668562723            8      rockstar2297       MIDDLE   
198  EUW1_5668562723            9           Yazinto       BOTTOM   
199  EUW1_5668562723           10     Purple Dragon      UTILITY   

     championName     kda  champLevel  totalDamageToChampions  \
0           Fiora  11/2/3          16                   30651   
1         Karthus  4/6/11          15                

In [30]:
kelody = participantsDf[participantsDf["summonerName"] == "Kelody"][["matchId", "participant"]]
kelody

,matchId,participant
2,EUW1_5683109745,3
17,EUW1_5682792181,8
22,EUW1_5682732968,3
37,EUW1_5679823597,8
42,EUW1_5679723082,3
57,EUW1_5675482746,8
68,EUW1_5674228929,9
78,EUW1_5674281765,9
82,EUW1_5673562345,3
98,EUW1_5672496690,9


In [31]:
kelodyEvents = []

# (matchTimelineDf["assistingParticipantIds"] == kelody["participant"].values[0]
for index, match in kelody.iterrows():
    game = []
    assist = matchTimelineDf[(matchTimelineDf["matchId"] == match["matchId"]) & matchTimelineDf["assistingParticipantIds"].notna() & (matchTimelineDf["creatorId"] != match["participant"]) & (matchTimelineDf["killerId"] != match["participant"]) & (matchTimelineDf["victimId"] != match["participant"])]
    participantList = as_list(match["participant"])
    game = matchTimelineDf.loc[((matchTimelineDf["matchId"] == match["matchId"]) & ((matchTimelineDf["participantId"] == (match["participant"])) | 
    (matchTimelineDf["creatorId"] == match["participant"]) |
    (matchTimelineDf["killerId"] == match["participant"]) |
    (matchTimelineDf["victimId"] == match["participant"]) |
    (assist["assistingParticipantIds"].apply(lambda x: any(i in x for i in participantList)))))]
    kelodyEvents.append(game)
kelodyEventsDf = pd.concat(kelodyEvents, ignore_index=True)
kelodyEventsDf

,matchId,type,participantId,itemId,timestamp,wardType,creatorId,level,killerId,victimId,assistingParticipantIds,bounty,positionX,positionY,teamId,buildingType,laneType,monsterType
0,EUW1_5683109745,ITEM_PURCHASED,3.0,1054.0,8302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EUW1_5683109745,ITEM_PURCHASED,3.0,2003.0,8769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EUW1_5683109745,ITEM_PURCHASED,3.0,3340.0,8937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EUW1_5683109745,LEVEL_UP,3.0,NaN,144347,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EUW1_5683109745,LEVEL_UP,3.0,NaN,161552,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,EUW1_5668562723,CHAMPION_KILL,NaN,NaN,1565024,NaN,NaN,NaN,4.0,7.0,"[3, 5]",175.0,13005.0,11677.0,NaN,NaN,NaN,NaN
1452,EUW1_5668562723,CHAMPION_KILL,NaN,NaN,1593504,NaN,NaN,NaN,8.0,3.0,[9],300.0,11983.0,13978.0,NaN,NaN,NaN,NaN
1453,EUW1_5668562723,ITEM_PURCHASED,3.0,3145.0,1596214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1454,EUW1_5668562723,ITEM_PURCHASED,3.0,3041.0,1637732,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check how often a Player died in Early Game (pre 10 mins)

In [32]:
sumOfDeaths = []
for index, match in kelody.iterrows():
     deaths = len(kelodyEventsDf[(kelodyEventsDf["matchId"] == match["matchId"]) & (kelodyEventsDf["victimId"] == match["participant"]) & (kelodyEventsDf["timestamp"] <= 600000)].index)
     sumOfDeaths.insert(len(sumOfDeaths), deaths)
kelody["deathsPreMin10"] = sumOfDeaths
kelody

,matchId,participant,deathsPreMin10
2,EUW1_5683109745,3,0
17,EUW1_5682792181,8,2
22,EUW1_5682732968,3,4
37,EUW1_5679823597,8,3
42,EUW1_5679723082,3,1
57,EUW1_5675482746,8,2
68,EUW1_5674228929,9,2
78,EUW1_5674281765,9,2
82,EUW1_5673562345,3,1
98,EUW1_5672496690,9,0


In [ ]:
print(timeline['info']['frames'][1])
participantsDf.to_json('participants.json', orient='records')
matchesDf.to_json('matches.json', orient='records')
matchTimelineDf.to_json('matchTimeline.json', orient='records')
